In [106]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


**Housing Prices Competition - Advanced Regression Techniques**
https://www.kaggle.com/c/house-prices-advanced-regression-techniques

In [107]:

train_d_full = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
train_d_full.head()


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [108]:
train_d_full.shape
# Rows x Columns
print(train_d_full['SalePrice'].mean())

180921.19589041095


In [109]:
test_d = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")
test_d.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


Mean Sale Price (Target Feature) = $180,921

In [111]:
y = train_d_full.SalePrice
for i in train_d_full.columns:
    st = set(train_d_full[i])
    if(len(st) == 2):
        print(i)
        print(st)
cols_dropped = ['SalePrice', 'Utilities', 'Street']
#Utilities and Streets Dropped as they have no contribution to dictating House Prices
train_d = train_d_full.drop(cols_dropped, axis = 1)
cols_dropped_tst = ['Utilities', 'Street']
test_d = test_d.drop(cols_dropped_tst, axis = 1)

Street
{'Grvl', 'Pave'}
Utilities
{'NoSeWa', 'AllPub'}
CentralAir
{'Y', 'N'}


In [65]:
# Split training data into training and validation subsets
# Smaller Data set so will use cross validation after

In [112]:
from sklearn.model_selection import train_test_split
X = train_d
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

In [113]:
# Define Numerical Columns and Categorical Columns

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
# Keep all Columns
#categorical_cols = [cname for cname in train_d_full.columns if train_d_full[cname].nunique() < 10 and 
       #                 train_d_full[cname].dtype == "object"]'
categorical_cols = [cname for cname in train_X.columns if train_X[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in train_X.columns if train_X[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = train_X[my_cols].copy()
X_val = val_X[my_cols].copy()

*PreProcessing Below (Using Pipeline) : *


In [114]:
# Columns of Categorical Data
c = (train_X.dtypes == 'object')
object_cols = list(c[c].index)


from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='mean')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [115]:
# Missing Data
cols_missing = [col for col in train_X.columns
                     if train_X[col].isnull().any()]
print(cols_missing)

['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']


In [119]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
# Parameter Tuning
# Baseline MAE :  $ 17293.552183219177
# Mean NMAE Scores :  $ 18295.807229238017

# Tune n_estimators and learning rate until flat line score reached

# Use Grid Search to optimize model and find best (smallest) mae on validation data 
model = XGBRegressor(n_estimators = 600, learning_rate = 0.05)

pipeline_fin = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

pipeline_fin.fit(X_train, train_y,)


predictions = pipeline_fin.predict(X_val)
print('MAE XGBoost : ', mean_absolute_error(predictions, val_y))

# Cross Validation Scores Below : 
scores = -1 * cross_val_score(pipeline_fin, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')
print('Cross Validation : ', list(scores))
print('Mean Cross NMAE Scores : ', scores.mean())


MAE XGBoost :  16325.107266695206
Cross Validation :  [15651.459452589897, 16730.43820901113, 16846.502541738013, 14544.441647046233, 20417.671446917808]
Mean Cross NMAE Scores :  16838.102659460616


In [117]:
# Make CSV file for submission

# Final Predictions :
print(len(X.columns))
pipeline_fin.fit(X,y)
preds_test = pipeline_fin.predict(test_d)
output = pd.DataFrame({'Id': test_d.Id, 'SalePrice': preds_test})
output.to_csv('submission_hp_first.csv', index = False)

print("Your submission was successfully saved!")

78
Your submission was successfully saved!
